# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 14 2022 11:09AM,241867,16,8273890,Sartorius Bioprocess Solutions,Released
1,Jul 14 2022 11:02AM,241863,10,0085873162,ISDIN Corporation,Released
2,Jul 14 2022 11:02AM,241863,10,0085873168,ISDIN Corporation,Released
3,Jul 14 2022 11:01AM,241862,10,MSP212138,"Methapharm, Inc.",Released
4,Jul 14 2022 11:01AM,241862,10,MSP212150,"Methapharm, Inc.",Released
5,Jul 14 2022 11:01AM,241862,10,MSP212151,"Methapharm, Inc.",Released
6,Jul 14 2022 11:01AM,241862,10,MSP212152,"Methapharm, Inc.",Released
7,Jul 14 2022 11:01AM,241862,10,MSP212153,"Methapharm, Inc.",Released
8,Jul 14 2022 11:01AM,241862,10,MSP212154,"Methapharm, Inc.",Released
9,Jul 14 2022 11:01AM,241862,10,MSP212156,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
39,241859,Released,3
40,241861,Released,1
41,241862,Released,10
42,241863,Released,2
43,241867,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
241859,NaN,NaN,3.0
241861,NaN,NaN,1.0
241862,NaN,NaN,10.0
241863,NaN,NaN,2.0
241867,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241771,0.0,1.0,0.0
241784,0.0,0.0,1.0
241785,0.0,1.0,0.0
241786,0.0,0.0,1.0
241787,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241771,0,1,0
241784,0,0,1
241785,0,1,0
241786,0,0,1
241787,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241771,0,1,0
1,241784,0,0,1
2,241785,0,1,0
3,241786,0,0,1
4,241787,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241771,,1,
1,241784,,,1
2,241785,,1,
3,241786,,,1
4,241787,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 14 2022 11:09AM,241867,16,Sartorius Bioprocess Solutions
1,Jul 14 2022 11:02AM,241863,10,ISDIN Corporation
3,Jul 14 2022 11:01AM,241862,10,"Methapharm, Inc."
13,Jul 14 2022 10:44AM,241861,10,"Citieffe, Inc."
14,Jul 14 2022 10:41AM,241859,15,"Virtus Pharmaceuticals, LLC"
17,Jul 14 2022 10:39AM,241857,10,Bio-PRF
18,Jul 14 2022 10:38AM,241858,19,ACG North America LLC
19,Jul 14 2022 10:35AM,241856,19,"GUSA Granules USA, Inc."
20,Jul 14 2022 10:30AM,241854,10,Beach Products Inc
34,Jul 14 2022 10:04AM,241853,10,Eywa Pharma Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 14 2022 11:09AM,241867,16,Sartorius Bioprocess Solutions,,,1
1,Jul 14 2022 11:02AM,241863,10,ISDIN Corporation,,,2
2,Jul 14 2022 11:01AM,241862,10,"Methapharm, Inc.",,,10
3,Jul 14 2022 10:44AM,241861,10,"Citieffe, Inc.",,,1
4,Jul 14 2022 10:41AM,241859,15,"Virtus Pharmaceuticals, LLC",,,3
5,Jul 14 2022 10:39AM,241857,10,Bio-PRF,,,1
6,Jul 14 2022 10:38AM,241858,19,ACG North America LLC,,,1
7,Jul 14 2022 10:35AM,241856,19,"GUSA Granules USA, Inc.",,,1
8,Jul 14 2022 10:30AM,241854,10,Beach Products Inc,,,14
9,Jul 14 2022 10:04AM,241853,10,Eywa Pharma Inc.,,,19


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 14 2022 11:09AM,241867,16,Sartorius Bioprocess Solutions,1,,
1,Jul 14 2022 11:02AM,241863,10,ISDIN Corporation,2,,
2,Jul 14 2022 11:01AM,241862,10,"Methapharm, Inc.",10,,
3,Jul 14 2022 10:44AM,241861,10,"Citieffe, Inc.",1,,
4,Jul 14 2022 10:41AM,241859,15,"Virtus Pharmaceuticals, LLC",3,,
5,Jul 14 2022 10:39AM,241857,10,Bio-PRF,1,,
6,Jul 14 2022 10:38AM,241858,19,ACG North America LLC,1,,
7,Jul 14 2022 10:35AM,241856,19,"GUSA Granules USA, Inc.",1,,
8,Jul 14 2022 10:30AM,241854,10,Beach Products Inc,14,,
9,Jul 14 2022 10:04AM,241853,10,Eywa Pharma Inc.,19,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 14 2022 11:09AM,241867,16,Sartorius Bioprocess Solutions,1,,
1,Jul 14 2022 11:02AM,241863,10,ISDIN Corporation,2,,
2,Jul 14 2022 11:01AM,241862,10,"Methapharm, Inc.",10,,
3,Jul 14 2022 10:44AM,241861,10,"Citieffe, Inc.",1,,
4,Jul 14 2022 10:41AM,241859,15,"Virtus Pharmaceuticals, LLC",3,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 14 2022 11:09AM,241867,16,Sartorius Bioprocess Solutions,1.0,NaN,NaN
1,Jul 14 2022 11:02AM,241863,10,ISDIN Corporation,2.0,NaN,NaN
2,Jul 14 2022 11:01AM,241862,10,"Methapharm, Inc.",10.0,NaN,NaN
3,Jul 14 2022 10:44AM,241861,10,"Citieffe, Inc.",1.0,NaN,NaN
4,Jul 14 2022 10:41AM,241859,15,"Virtus Pharmaceuticals, LLC",3.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 14 2022 11:09AM,241867,16,Sartorius Bioprocess Solutions,1.0,0.0,0.0
1,Jul 14 2022 11:02AM,241863,10,ISDIN Corporation,2.0,0.0,0.0
2,Jul 14 2022 11:01AM,241862,10,"Methapharm, Inc.",10.0,0.0,0.0
3,Jul 14 2022 10:44AM,241861,10,"Citieffe, Inc.",1.0,0.0,0.0
4,Jul 14 2022 10:41AM,241859,15,"Virtus Pharmaceuticals, LLC",3.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3869496,186.0,0.0,0.0
12,725468,15.0,26.0,0.0
15,241859,3.0,0.0,0.0
16,1692731,6.0,2.0,0.0
19,967351,31.0,24.0,10.0
20,725481,24.0,5.0,0.0
21,1208999,5.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3869496,186.0,0.0,0.0
1,12,725468,15.0,26.0,0.0
2,15,241859,3.0,0.0,0.0
3,16,1692731,6.0,2.0,0.0
4,19,967351,31.0,24.0,10.0
5,20,725481,24.0,5.0,0.0
6,21,1208999,5.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,186.0,0.0,0.0
1,12,15.0,26.0,0.0
2,15,3.0,0.0,0.0
3,16,6.0,2.0,0.0
4,19,31.0,24.0,10.0
5,20,24.0,5.0,0.0
6,21,5.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,186.0
1,12,Released,15.0
2,15,Released,3.0
3,16,Released,6.0
4,19,Released,31.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,0.0,0.0,0.0,0.0,10.0,0.0,0.0
Executing,0.0,26.0,0.0,2.0,24.0,5.0,0.0
Released,186.0,15.0,3.0,6.0,31.0,24.0,5.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,0.0,0.0,0.0,0.0,10.0,0.0,0.0
1,Executing,0.0,26.0,0.0,2.0,24.0,5.0,0.0
2,Released,186.0,15.0,3.0,6.0,31.0,24.0,5.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,0.0,0.0,0.0,0.0,10.0,0.0,0.0
1,Executing,0.0,26.0,0.0,2.0,24.0,5.0,0.0
2,Released,186.0,15.0,3.0,6.0,31.0,24.0,5.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()